# Benders decomposition using Julia
## Nondecomposed problem solution

In [1]:
using JuMP, OSQP, GLPK

In [2]:
using Printf, Plots

In [3]:
## Parameters
# costs
cS1 = [10,20]
cS2 = [20,80]

# scenario probability
pi_w = [0.25,0.25,0.25,0.25]

# uncertain parameter
theta = [30,60,70,10]
theta_lb = [0,0,0,0];

In [4]:
## Modeling the non decomposed problem  --> checking the problem use> print(master)
nonDec = Model(GLPK.Optimizer)

#############
# VARIABLES #
#############
@variable(nonDec, 0 ≤ p_da_g1 ≤ 100, start = 90)
@variable(nonDec, 0 ≤ p_da_g2 ≤ 30, start = 30)
@variable(nonDec, 0 ≤ p_da_w ≤ 70, start = 0)
# scenario 1:4
@variable(nonDec, -30 ≤ p_rt_g2_s[1:4] ≤ 30)
@variable(nonDec, 0 ≤ p_curtail_s[1:4] ≤ 120)
@variable(nonDec , 0 <= p_spill_s[i=1:4] <= theta[i])

###############
# CONSTRAINTS #
###############
@constraint(nonDec, p_da_g1 + p_da_g2 + p_da_w == 120)
# scenario 1
@constraint(nonDec, s1[i=1:4], 0 ≤ p_da_g2 + p_rt_g2_s[i] ≤ 30)
@constraint(nonDec, s2[i=1:4], p_rt_g2_s[i] + (theta[i] -  p_da_w - p_spill_s[i]) + p_curtail_s[i] == 0)

@objective(nonDec, Min, cS1[1]*p_da_g1 + cS1[2]*p_da_g2 + 0.25*(cS2[1]*sum(p_rt_g2_s[i] for i in 1:4) + cS2[2]*sum(p_curtail_s[i] for i in 1:4)));

In [5]:
# print(nonDec)

In [6]:
function print_iteration(k, args...)
    f(x) = Printf.@sprintf("%12.4e",x)
    println(lpad(k,9), " ",join(f.(args), " "))
    return
end

print_iteration (generic function with 1 method)

In [7]:
optimize!(nonDec)

println(" **** solution found ********")
# lower bound of the original problem
o = objective_value(nonDec)
p_g1_da = value(p_da_g1)
p_g2_da = value(p_da_g2)
p_w_da = value(p_da_w)

print_iteration(0,o)
print_iteration(0,p_g1_da,p_g2_da,p_w_da)

for wi in 1:4
    rt = value(p_rt_g2_s[wi])
    spill = value(p_spill_s[wi])
    curt = value(p_curtail_s[wi])
    print_iteration(wi,rt,spill,curt)
end

#all_con = all_constraints(nonDec, AffExpr, MOI.EqualTo{Float64})
#print(all_con)
#λ = dual.(all_con)

 **** solution found ********
        0   1.0000e+03
        0   9.0000e+01   3.0000e+01   0.0000e+00
        1  -3.0000e+01   0.0000e+00   0.0000e+00
        2  -3.0000e+01   3.0000e+01   0.0000e+00
        3  -3.0000e+01   4.0000e+01   0.0000e+00
        4  -1.0000e+01   0.0000e+00   0.0000e+00
